# Final small business loan approval  

This notebook marks the final loan approval work around for my final year capstone project.

## problem definition:
The case-study project: we assume the role of loan officer at a bank and try to approve or deny a loan by assessing its risk of default using machine learning models. The most accurate model will be picked for deployment and integration with an interface for interactive decision making.

## project objective:
The project will try to answer the following questions: As a representative of the bank, should I grant a loan to a particular small business (Company X)? Why or why not? The decision made by assessing a loan's risk.

generally:
The assessment is accomplished by estimating the loan's default probability through analyzing the historical dataset and then classifying the loan into one of two categories: (a) higher risk—likely to default on the loan (i.e., be charged off/failure to pay in full) or (b) lower risk—likely to pay off the loan in full. There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans. 

## Dataset:
The dataset used in this project is the U.S. Small Business Administration (SBA). Here is the link to the dataset- [Link Text](https://www.kaggle.com/datasets/mirbektoktogaraev/should-this-loan-be-approved-or-denied) 

The dataset is a real dataset from the U.S. Small Business Administration (SBA). The case-study assignment, titled “Should This Loan be Approved or Denied?” is designed to teach statistical thinking by focusing on how to use real data to make informed decisions for a particular purpose. For this assignment, students assume the role of a loan officer who is deciding whether to approve a loan to a small business.

In [2]:
# packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

# we want plots to appear inside the notebook
%matplotlib inline

# Models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XCBClassifier

# Model evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve

# imports for column transformations
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Pipeline and feature selection
from sklearn.pipeline import Pipeline

In [59]:
import random
import json
from loguru import logger

# Loan Data

In [9]:
# this code ignores all the warnings 
import warnings
warnings.filterwarnings("ignore") # you can change "igonre" to "default"

In [3]:
# Import Dataset
sbanational_df = pd.read_csv('data/SBAnational/SBAnational.csv', low_memory=False)
sbanational_df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [4]:
sbanational_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [5]:
sbanational_df.MIS_Status.value_counts()

MIS_Status
P I F     739609
CHGOFF    157558
Name: count, dtype: int64

In [7]:
sbanational_df.shape

(899164, 27)

In [8]:
# check for duplicates
sbanational_df.duplicated().any()

False

# Clean data

In [10]:
sbanational_df.head().T

,0,1,2,3,4
LoanNr_ChkDgt,1000014003,1000024006,1000034009,1000044001,1000054004
Name,ABC HOBBYCRAFT,LANDMARK BAR & GRILLE (THE),"WHITLOCK DDS, TODD M.","BIG BUCKS PAWN & JEWELRY, LLC","ANASTASIA CONFECTIONS, INC."
City,EVANSVILLE,NEW PARIS,BLOOMINGTON,BROKEN ARROW,ORLANDO
State,IN,IN,IN,OK,FL
Zip,47711,46526,47401,74012,32801
Bank,FIFTH THIRD BANK,1ST SOURCE BANK,GRANT COUNTY STATE BANK,1ST NATL BK & TR CO OF BROKEN,FLORIDA BUS. DEVEL CORP
BankState,OH,IN,IN,OK,FL
NAICS,451120,722410,621210,0,0
ApprovalDate,28-Feb-97,28-Feb-97,28-Feb-97,28-Feb-97,28-Feb-97
ApprovalFY,1997,1997,1997,1997,1997


In [11]:
# check for missing values
sbanational_df.isna().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

In [16]:
len(sbanational_df["ChgOffDate"])

899164

In [17]:
# check percentage of missing values on ChgOffPrinGr
sbanational_df["ChgOffDate"].isna().sum()/len(sbanational_df["ChgOffDate"]) 

0.8190552557709161

In [20]:
# check percentage of missing values on all columns

def check_percentage_of_missing_data(df):
    total_rows = len(df)
    missing_percentage = (df.isna().sum() / total_rows)*100

    missing_dict = pd.DataFrame({
        "Column Name:": missing_percentage.index,
        "Percrentage of missing Data:": missing_percentage.values
    })

    print(missing_dict.to_string(index=False))

In [21]:
check_percentage_of_missing_data(sbanational_df)

     Column Name:  Percrentage of missing Data:
    LoanNr_ChkDgt                      0.000000
             Name                      0.001557
             City                      0.003336
            State                      0.001557
              Zip                      0.000000
             Bank                      0.173383
        BankState                      0.174162
            NAICS                      0.000000
     ApprovalDate                      0.000000
       ApprovalFY                      0.000000
             Term                      0.000000
            NoEmp                      0.000000
         NewExist                      0.015125
        CreateJob                      0.000000
      RetainedJob                      0.000000
    FranchiseCode                      0.000000
       UrbanRural                      0.000000
        RevLineCr                      0.503579
           LowDoc                      0.287156
       ChgOffDate                     81

In [22]:
# drop ChgOffDate
sbanational_df.drop('ChgOffDate', axis = 1, inplace=True)

In [24]:
# check for NaN values
sbanational_df.isna().sum()

LoanNr_ChkDgt           0
Name                   14
City                   30
State                  14
Zip                     0
Bank                 1559
BankState            1566
NAICS                   0
ApprovalDate            0
ApprovalFY              0
Term                    0
NoEmp                   0
NewExist              136
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr            4528
LowDoc               2582
DisbursementDate     2368
DisbursementGross       0
BalanceGross            0
MIS_Status           1997
ChgOffPrinGr            0
GrAppv                  0
SBA_Appv                0
dtype: int64

In [26]:
# drop all rows with missing values
sbanational_df.dropna(inplace=True)

In [27]:
# check if we still have missing values
sbanational_df.isna().sum()

LoanNr_ChkDgt        0
Name                 0
City                 0
State                0
Zip                  0
Bank                 0
BankState            0
NAICS                0
ApprovalDate         0
ApprovalFY           0
Term                 0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementDate     0
DisbursementGross    0
BalanceGross         0
MIS_Status           0
ChgOffPrinGr         0
GrAppv               0
SBA_Appv             0
dtype: int64

In [28]:
sbanational_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 886240 entries, 0 to 899163
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      886240 non-null  int64  
 1   Name               886240 non-null  object 
 2   City               886240 non-null  object 
 3   State              886240 non-null  object 
 4   Zip                886240 non-null  int64  
 5   Bank               886240 non-null  object 
 6   BankState          886240 non-null  object 
 7   NAICS              886240 non-null  int64  
 8   ApprovalDate       886240 non-null  object 
 9   ApprovalFY         886240 non-null  object 
 10  Term               886240 non-null  int64  
 11  NoEmp              886240 non-null  int64  
 12  NewExist           886240 non-null  float64
 13  CreateJob          886240 non-null  int64  
 14  RetainedJob        886240 non-null  int64  
 15  FranchiseCode      886240 non-null  int64  
 16  UrbanRu

In [35]:
date_columns = sbanational_df.select_dtypes(include=['datetime64'])
print(date_columns.columns.tolist())

[]


In [36]:
# convert date cols to date format
date_columns=['ApprovalDate','DisbursementDate']
sbanational_df[date_columns] = sbanational_df[date_columns].apply(pd.to_datetime)

# print columns
date_columns = sbanational_df.select_dtypes(include=['datetime64'])
print(date_columns.columns.tolist())

['ApprovalDate', 'DisbursementDate']


In [37]:
sbanational_df.head().T

,0,1,2,3,4
LoanNr_ChkDgt,1000014003,1000024006,1000034009,1000044001,1000054004
Name,ABC HOBBYCRAFT,LANDMARK BAR & GRILLE (THE),"WHITLOCK DDS, TODD M.","BIG BUCKS PAWN & JEWELRY, LLC","ANASTASIA CONFECTIONS, INC."
City,EVANSVILLE,NEW PARIS,BLOOMINGTON,BROKEN ARROW,ORLANDO
State,IN,IN,IN,OK,FL
Zip,47711,46526,47401,74012,32801
Bank,FIFTH THIRD BANK,1ST SOURCE BANK,GRANT COUNTY STATE BANK,1ST NATL BK & TR CO OF BROKEN,FLORIDA BUS. DEVEL CORP
BankState,OH,IN,IN,OK,FL
NAICS,451120,722410,621210,0,0
ApprovalDate,1997-02-28 00:00:00,1997-02-28 00:00:00,1997-02-28 00:00:00,1997-02-28 00:00:00,1997-02-28 00:00:00
ApprovalFY,1997,1997,1997,1997,1997


In [39]:
# clean all currency columns
currency_columns = ['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']
sbanational_df[currency_columns] = sbanational_df[currency_columns].replace('[\$,]', '', regex=True).astype(float)

In [40]:
sbanational_df.head().T

,0,1,2,3,4
LoanNr_ChkDgt,1000014003,1000024006,1000034009,1000044001,1000054004
Name,ABC HOBBYCRAFT,LANDMARK BAR & GRILLE (THE),"WHITLOCK DDS, TODD M.","BIG BUCKS PAWN & JEWELRY, LLC","ANASTASIA CONFECTIONS, INC."
City,EVANSVILLE,NEW PARIS,BLOOMINGTON,BROKEN ARROW,ORLANDO
State,IN,IN,IN,OK,FL
Zip,47711,46526,47401,74012,32801
Bank,FIFTH THIRD BANK,1ST SOURCE BANK,GRANT COUNTY STATE BANK,1ST NATL BK & TR CO OF BROKEN,FLORIDA BUS. DEVEL CORP
BankState,OH,IN,IN,OK,FL
NAICS,451120,722410,621210,0,0
ApprovalDate,1997-02-28 00:00:00,1997-02-28 00:00:00,1997-02-28 00:00:00,1997-02-28 00:00:00,1997-02-28 00:00:00
ApprovalFY,1997,1997,1997,1997,1997


# Feature Engineering

For this project we need the Industry Column ,
the NAICS column has the industry code. The fisrt two digits indicate a company's industry

In [41]:
sbanational_df['Industry'] = sbanational_df['NAICS'].astype('str').apply(lambda x: x[:2])
sbanational_df['Industry'] = sbanational_df['Industry'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin',
    '0': 'Other'
})

In [42]:
sbanational_df.head().T

,0,1,2,3,4
LoanNr_ChkDgt,1000014003,1000024006,1000034009,1000044001,1000054004
Name,ABC HOBBYCRAFT,LANDMARK BAR & GRILLE (THE),"WHITLOCK DDS, TODD M.","BIG BUCKS PAWN & JEWELRY, LLC","ANASTASIA CONFECTIONS, INC."
City,EVANSVILLE,NEW PARIS,BLOOMINGTON,BROKEN ARROW,ORLANDO
State,IN,IN,IN,OK,FL
Zip,47711,46526,47401,74012,32801
Bank,FIFTH THIRD BANK,1ST SOURCE BANK,GRANT COUNTY STATE BANK,1ST NATL BK & TR CO OF BROKEN,FLORIDA BUS. DEVEL CORP
BankState,OH,IN,IN,OK,FL
NAICS,451120,722410,621210,0,0
ApprovalDate,1997-02-28 00:00:00,1997-02-28 00:00:00,1997-02-28 00:00:00,1997-02-28 00:00:00,1997-02-28 00:00:00
ApprovalFY,1997,1997,1997,1997,1997


# EDA

# Select Suitable Features

In [49]:
relevant_columns=['GrAppv', 'Term', 'NoEmp','NewExist', 'UrbanRural', 'Industry','MIS_Status']
df=sbanational_df.loc[:,relevant_columns]
df

,GrAppv,Term,NoEmp,NewExist,UrbanRural,Industry,MIS_Status
0,60000.0,84,4,2.0,0,Retail_trade,P I F
1,40000.0,60,2,2.0,0,Accom/Food_serv,P I F
2,287000.0,180,7,1.0,0,Healthcare/Social_assist,P I F
3,35000.0,60,2,1.0,0,Other,P I F
4,229000.0,240,14,1.0,0,Other,P I F
...,...,...,...,...,...,...,...
899159,70000.0,60,6,1.0,0,Retail_trade,P I F
899160,85000.0,60,6,1.0,0,Retail_trade,P I F
899161,300000.0,108,26,1.0,0,Manufacturing,P I F
899162,75000.0,60,6,1.0,0,Other,CHGOFF


# Transform Data

preprocessing stage

In [43]:
# Extract all non-numeric columns
non_numeric_columns = sbanational_df.select_dtypes(include=['object'])
non_numeric_columns.head()

,Name,City,State,Bank,BankState,ApprovalFY,RevLineCr,LowDoc,MIS_Status,Industry
0,ABC HOBBYCRAFT,EVANSVILLE,IN,FIFTH THIRD BANK,OH,1997,N,Y,P I F,Retail_trade
1,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,1ST SOURCE BANK,IN,1997,N,Y,P I F,Accom/Food_serv
2,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,GRANT COUNTY STATE BANK,IN,1997,N,N,P I F,Healthcare/Social_assist
3,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,1ST NATL BK & TR CO OF BROKEN,OK,1997,N,Y,P I F,Other
4,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,FLORIDA BUS. DEVEL CORP,FL,1997,N,N,P I F,Other


In [50]:
column_names = non_numeric_columns.columns.tolist()
print(column_names)

['Name', 'City', 'State', 'Bank', 'BankState', 'ApprovalFY', 'RevLineCr', 'LowDoc', 'MIS_Status', 'Industry']


In [51]:
features = non_numeric_columns.columns.to_numpy()
features

array(['Name', 'City', 'State', 'Bank', 'BankState', 'ApprovalFY',
       'RevLineCr', 'LowDoc', 'MIS_Status', 'Industry'], dtype=object)

In [53]:
# Encoding DataFrame
encoded_df=pd.DataFrame()
for column in df.columns:
    if df[column].dtype == 'object':
        encoded_df[column] = df[column].astype('category').cat.codes
    else:
        encoded_df[column] = df[column]

In [54]:
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 886240 entries, 0 to 899163
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   GrAppv      886240 non-null  float64
 1   Term        886240 non-null  int64  
 2   NoEmp       886240 non-null  int64  
 3   NewExist    886240 non-null  float64
 4   UrbanRural  886240 non-null  int64  
 5   Industry    886240 non-null  int8   
 6   MIS_Status  886240 non-null  int8   
dtypes: float64(2), int64(3), int8(2)
memory usage: 42.3 MB


# Training 

In [55]:
# Split the data
np.random.seed(42)

X = encoded_df.drop("MIS_Status", axis=1)
y = encoded_df["MIS_Status"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Modeling

### Expected Output
1. Whether loan is guaranteed or not
2. Major factors that led to loan results
3. Most Probable Banks

### Random Forest

In [56]:
# initialize model
rf_classifier = RandomForestClassifier()

# fit the model
rf_classifier.fit(X_train, y_train)

RandomForestClassifier()

In [57]:
# check model score
rf_classifier.score(X_test, y_test)

0.9213531323343563

# Incorporate external data


This is a very crucial part of the project:

By incoporating external information the project ensures improvement in the model's ability to keep up with external factors that may affect the overall loan decision. This will reduce errors in the model's prediction ability. This segment will add `Industry trends` as an external factor to drive the overall decision of this model. 

Industry trends values:
1. Positive 
2. Negative
3. Neutral

### Engineer industry trends column

In [58]:
# column to be created
column_to_move = 'Industry Trends'
last_index = len(sbanational_df.columns) - 1

sbanational_df.insert(last_index, column_to_move, "")
sbanational_df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,Industry Trends,Industry
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,1997-02-28,1997,...,Y,1999-02-28,60000.0,0.0,P I F,0.0,60000.0,48000.0,,Retail_trade
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,1997-02-28,1997,...,Y,1997-05-31,40000.0,0.0,P I F,0.0,40000.0,32000.0,,Accom/Food_serv
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,1997-02-28,1997,...,N,1997-12-31,287000.0,0.0,P I F,0.0,287000.0,215250.0,,Healthcare/Social_assist
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,1997-02-28,1997,...,Y,1997-06-30,35000.0,0.0,P I F,0.0,35000.0,28000.0,,Other
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,1997-02-28,1997,...,N,1997-05-14,229000.0,0.0,P I F,0.0,229000.0,229000.0,,Other


### random sentiments 

In [60]:
# industries

# Dictionary mapping for industries
industry_mapping = {
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin',
}

# Extract the values from the dictionary
industry_array = list(industry_mapping.values())

print("Industries array:")
print(industry_array)

Industries array:
['Ag/For/Fish/Hunt', 'Min/Quar/Oil_Gas_ext', 'Utilities', 'Construction', 'Manufacturing', 'Manufacturing', 'Manufacturing', 'Wholesale_trade', 'Retail_trade', 'Retail_trade', 'Trans/Ware', 'Trans/Ware', 'Information', 'Finance/Insurance', 'RE/Rental/Lease', 'Prof/Science/Tech', 'Mgmt_comp', 'Admin_sup/Waste_Mgmt_Rem', 'Educational', 'Healthcare/Social_assist', 'Arts/Entertain/Rec', 'Accom/Food_serv', 'Other_no_pub', 'Public_Admin']


In [61]:
# create random sentiments for training purposes

industry_sentiments_random = {}

for sector in industry_array:
    sentiment = random.choice(['positive', 'neutral', 'negative'])
    industry_sentiments_random[sector] = sentiment

logger.info(f"{json.dumps(industry_sentiments_random, indent=4)}")

2024-04-27 19:35:04.499 | INFO     | __main__:<module>:9 - {
    "Ag/For/Fish/Hunt": "negative",
    "Min/Quar/Oil_Gas_ext": "neutral",
    "Utilities": "positive",
    "Construction": "neutral",
    "Manufacturing": "positive",
    "Wholesale_trade": "negative",
    "Retail_trade": "neutral",
    "Trans/Ware": "negative",
    "Information": "positive",
    "Finance/Insurance": "positive",
    "RE/Rental/Lease": "negative",
    "Prof/Science/Tech": "positive",
    "Mgmt_comp": "neutral",
    "Admin_sup/Waste_Mgmt_Rem": "negative",
    "Educational": "negative",
    "Healthcare/Social_assist": "positive",
    "Arts/Entertain/Rec": "neutral",
    "Accom/Food_serv": "positive",
    "Other_no_pub": "neutral",
    "Public_Admin": "neutral"
}


### populate Industry trends

using random trends

In [ ]:
new_df = sbanational_df
for industry, sentiment in industry_sentiments_random.items():
    # Populate the overall sentiment in the dataset for the current industry
    new_df.loc[new_df['Industry Sector'] == industry, 'Industry Trends'] = sentiment

populated_loan_random_dataset